# 10. Recursion

## Searching for Paths in a Labyrinth
### Our $\text{low-key}$ First Real Lesson in Game Design
---

Let's suppose we are given a **labyrinth** with a *rectangular* shape, consisting of $N \times M$ "squares".    

Each "square" is either *passable* or *impassable*.
      
An adventurer enters the labyrinth from its *top left corner* (the **start**) and has to **reach the bottom right corner** of the labyrinth (the **end**).    


<img src="_img/LabryinthOverview.jpg" style="display: block; margin: auto; width: 400px;"></img>

   
At each turn, the adventurer can move $\text{one position}$ $up$, $down$, $left$, or $right$, but may not go outside the boundaries of the labyrinth, or step on an impassable "square". 
   
It is also **prohibited to revisit the same "square"**.

<br>

To translate this concept into C# terms, we are going to use a **two-dimensional** `Array` of **characters**.    
   
In it we are going to mark the *passable* positions with the character `'_'` (**underscore**), the **end** of the labyrinth with the `e` character, and  the *impassable* positions with `*` character.  
   
The **starting position** is marked as *passable* position.    

In [1]:
// A Two Dimensional Character Array representation of the Labryinth defined above
char[ , ] Labryinth =
{
    //       M Columns             BY
    {'_','_','_','*','_','_','_'}, //
    {'*','*','_','*','_','*','_'}, //
    {'_','_','_','_','_','_','_'}, //      N Rows
    {'_','*','*','*','*','*','_'}, //
    {'_','_','_','_','_','_','e'}, //
};

<br>

Further, in accordance with the the example above,   
the $\text{number of rows}$, corresponding to $N$, is accesible in C# by way of the built-in Multidimensional Array method `GetLength( 0 )`, while the $\text{number of columns}$ corresponding to $M$, is subsequently accesible in C# by way of `GetLength( 1 )`. 

In [2]:
int N = Labryinth.GetLength( 0 ),
    M = Labryinth.GetLength( 1 );

<br>

To assist in visualizing our **two dimensional** `Labryinth`, we may define the following method for iteratively printing the full $M \times N$ grid of "squares" which it is made up of:  

In [3]:
// A helper method which prints out the 2D char array representation
// of the Labryinth
void DisplayLabrynth( char[ , ] labryinth )
{

    // Observe the First Row and Last Row Indices of the labryinth,
    // along with the First Column and Last Column Indices of the labryinth
    int FirstRowIndex = 0,
        FirstColIndex = 0,
        LastRowIndex  = labryinth.GetLength( 0 ) - 1,
        LastColIndex  = labryinth.GetLength( 1 ) - 1;


    // Iterating For each of the elements from the First Row Index of the labryinth 
    // up until the Last Row Index:
    for( int CurrentRowIndex = FirstRowIndex; CurrentRowIndex <= LastRowIndex; CurrentRowIndex++ )
    {

        // Iterating For each of the elements from the First Column Index of the labryinth 
        // up until the Last Column Index:
        for( int CurrentColIndex = FirstColIndex; CurrentColIndex <= LastColIndex; CurrentColIndex++ )
        {

            // Print the Value at the Current Col Index
            Console.Write(
                "{0,-3}", 
                $"{ labryinth[ CurrentRowIndex ,  CurrentColIndex ] } "
            );

        }

        // After all that,
        // each of the Column Values of the Current Row have been printed,
        // so we now move on, skipping the next two lines
        Console.WriteLine();

    }

    // After All that,
    // All Column Values have now been printed for every Row in the Array,
    // as required.

    // But skip an extra line though...
    Console.WriteLine();
    
}

In [4]:
DisplayLabrynth( Labryinth );

_  _  _  *  _  _  _  
*  *  _  *  _  *  _  
_  _  _  _  _  _  _  
_  *  *  *  *  *  _  
_  _  _  _  _  _  e  



<br>

### Objective

> **Print all possible paths** from the **start** of the labyrinth, to the **end**.

<br>

Below, you can see that there are **3 different paths** from the **starting** position to the **end** which meet the requirements of the task (movement only on *passable* squares and *not passing through any of the same squares*):

<img src="_img/LabryinthSolution.jpg" style="display: block; margin: auto"></img>

<br>

### Intuition for a Recursive Algorithm

We can consider searching *from* a given position in the labyrinth (including , but not limited to, the **start**), *to* the **end** of the labyrinth, as a **recursive** process: 
- Let the **current position** in the labyrinth be `(row, col)`.
  - In the beginning we go from the **start** position `(0, 0)`.
- If the **current position** is the **end** position, `(N-1, M-1)`, then we have found a path and we should print it.
- If the **current position** is *impassable*, we go back (we have no right to step on it).
- If the **current position** is *already visited*, we go back (we have no right to step on it twice).
- Otherwise, we look for a path in **four possible directions**. We search **recursively** for a path to the **end** from the labyrinth by trying to go in **all possible directions**: 
  - We try $left: \qquad \,\,\,$  position `(row, col - 1)` 
  - We try $up: \qquad\,\,\,\,\,\,\,$    position `(row - 1, col)`
  - We try $right: \qquad$ position `(row, col + 1)`
  - We try $down: \qquad$  position `(row + 1, col)`

<br>

#### Saving the Paths

In order to print the paths found by our recursive algorithm, we can use a **One-Dimensional** `Array` to store every step we take using the following **characters**: 
- `L` – left, 
- `U` – up, 
- `R` – right, 
- `D` – down
    
This `Array` will store each sequence of steps along the current path from the start of the labyrinth, up until to the current position.    
   
We are going to need an `Array` of type `char` and an *counter* to count the steps we have taken. 
   
The counter will keep track of how many times we have moved to the next position recursively, i.e. the current **depth** of recursion. 
   
In order to work correctly, our program has to *increment the counter* when entering recursion, and *store the direction* we have taken to get to the current position in the `Array`. 
   
When returning from a recursion, the counter should be *reduced by 1*. 
   
When an exit is found, the *path*, consisting of *all the characters* in the `Array` from 0 to the current position) should be printed. 

<br>

   
##### What should be the **size** of the **Path** `Array`?   
    
The answer to this question is easy; since we can enter each "square" at most once, then the path can never be longer than the *count of all squares*, which is equivalent to $N \times M$.    

In our case the **size** of the maze is $7 \times 5$, i.e. the **size** of the `Array` has to be `35`:

In [5]:
N * M

35

<br>

As such, we may declare the following **one-dimensional** `Array` of type `char` which will store each respective Path to the **end**: 

In [6]:
char[] Path =  new char[ N * M ];

<br>

We must also set up an **Path Index** to help iterate through the **Path** `Array` 

In [7]:
int PathIndex = 0;

<br>

Further, we may declare the following method to assist in printing out the current `Array` of *directional steps* which copmrise the **Path** :

In [8]:
void PrintPath( char[] path, int start, int end )
{
    Console.WriteLine("Found a Path to the end: ");

    // Iterating For every Current Index from the start, up until the end:
    for( int CurrentIndex = start; CurrentIndex <= end; CurrentIndex++ )
    {

        // Print the directional step at the Current Index
        Console.Write( path[ CurrentIndex ] );

    }
    
    // After all that,
    // We have printed each directional step from the start until the end.
    // We now print a line break to provide seperation for the next output.
    Console.WriteLine();

}

<br>

### Recursive Algorithm Implementation

Having performed the steps outlined above, we may now finally implement our recursive algorithm to **print all possible paths from the start of the labyrinth, to the end**. 

In [9]:
void FindPath( int row, int column, char directional_step )
{

    // If EITHER the given row or column arguments
    // are out of bounds of the Labryinth, which is an N X M character array:
    if( (row < 0 || column < 0) || ( row >= N || column >= M) )
    {

        // We are out of the the Labryinth
        return;

    }


    // Otherwise,
    // Append the current directional step to the Path array
    // taking care to simultaneously increment the Path Index
    Path[ PathIndex++ ] = directional_step;


    // If we have arrived at the end of the Labryinth:
    if( Labryinth[ row, column ] == 'e')
    {

        // Print out the Path of directional steps leading
        // to the end of the Labryinth
        PrintPath( Path, 1,  PathIndex - 1);
        DisplayLabrynth( Labryinth );

    }


    // If the Current "square" is passable:
    if( Labryinth[ row, column ] == '_' )
    {
    
        // Mark the Current "square" as being visited with a 'v'
        Labryinth[ row, column ] = 'v';


        // Recur in all directions relative to the current "square"
        FindPath( row,     column - 1, 'L');  //  left      
        FindPath( row - 1, column,     'U');  //  up      
        FindPath( row,     column + 1, 'R');  //  right      
        FindPath( row + 1, column,     'D');  //  down


        // Reset the Current "square" as free
        Labryinth[ row, column ] = '_';      

    }


    // Decrement the Path Index
    // to avoid going out of bounds ot the Path array 
    PathIndex--;

}

<br>

Now, all that remains to do is to simply invoke the recursive method which we have declared above, starting the first *visited* "square" at position `(0,0)`.

In [10]:
FindPath( 0, 0, 'v' );

Found a Path to the end: 
RRDDLLDDRRRRRR
v  v  v  *  _  _  _  
*  *  v  *  _  *  _  
v  v  v  _  _  _  _  
v  *  *  *  *  *  _  
v  v  v  v  v  v  e  

Found a Path to the end: 
RRDDRRUURRDDDD
v  v  v  *  v  v  v  
*  *  v  *  v  *  v  
_  _  v  v  v  _  v  
_  *  *  *  *  *  v  
_  _  _  _  _  _  e  

Found a Path to the end: 
RRDDRRRRDD
v  v  v  *  _  _  _  
*  *  v  *  _  *  _  
_  _  v  v  v  v  v  
_  *  *  *  *  *  v  
_  _  _  _  _  _  e  

